### Turtle to Dataframe

we import the libraries

In [1]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager


In [114]:
from IPython.display import display, HTML

In [8]:
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import pandas as pd

We open the browser and load the link

In [53]:

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
url = "https://www.scrapethissite.com/pages/frames/"
driver.get(url)

time.sleep(3)

get all the links for the turtles

In [54]:
iframe = driver.find_element(By.CSS_SELECTOR,'#iframe')

driver.switch_to.frame(iframe)

tortugas = driver.find_elements(By.CSS_SELECTOR,'.col-md-4.turtle-family-card')

enlaces= []

for tortuga in tortugas:
    
    #guardamos el enlace
    enlace = tortuga.find_element(By.CSS_SELECTOR,'a.btn.btn-default.btn-xs')
    enlace_url = enlace.get_attribute('href')

    enlaces.append(enlace_url)

print(enlaces)

['https://www.scrapethissite.com/pages/frames/?frame=i&family=Carettochelyidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Cheloniidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Chelydridae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Dermatemydidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Dermochelyidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Emydidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Geoemydidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Kinosternidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Platysternidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Testudinidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Trionychidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Chelidae', 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Pelomedusidae', 'https://w

In [55]:
print(len(enlaces))

14


scrape the data

In [56]:
cintific_names = []
common_names = []
text_tortus = []
photos_tortus = []

In [57]:

for enlace in enlaces:
    

    # Lo cargamos en una nueva ventana  
    driver.execute_script("window.open('" + enlace + "', '_blank');")

    # Cambiamos a esa ventana
    pagina_tortuga = driver.window_handles
    driver.switch_to.window(pagina_tortuga[-1]) 

    #Esperamos 
    time.sleep(1)


    # Scrapeamos los datos
    tortuga_name = driver.find_element(By.CSS_SELECTOR,'h3.family-name').text

    cintific_names.append(tortuga_name)
    
    common_name = driver.find_element(By.CSS_SELECTOR,'strong.common-name').text

    common_names.append(common_name)

    texto_tortu = driver.find_element(By.CSS_SELECTOR,'p.lead').text

    text_tortus.append(texto_tortu)


    # Conseguimos la imagen

    imagen_tortu = driver.find_element(By.CSS_SELECTOR,'img.turtle-image.center-block')
    imagen_tortu_enlace = imagen_tortu.get_attribute('src')

    photos_tortus.append(imagen_tortu_enlace)


    driver.close()

    driver.switch_to.window(pagina_tortuga[0]) 


driver.quit()

create the dataframe

In [72]:
tortugas= {
    'cientific name':cintific_names,
    'common name':common_names,
    'turtle text':text_tortus,
    'photo':photos_tortus,
}

df = pd.DataFrame(tortugas)

we see the text to extract more data

In [74]:
for i in df['turtle text']:
    print(i)

The Carettochelyidae family of turtles — more commonly known as "Pig-nosed turtle" — were first discovered in 1887 by Boulenger.
The Cheloniidae family of turtles — more commonly known as "Sea turtles" — were first discovered in 1811 by Oppel.
The Chelydridae family of turtles — more commonly known as "Snapping turtles" — were first discovered in 1831 by Gray.
The Dermatemydidae family of turtles — more commonly known as "Central American river turtle" — were first discovered in 1870 by Gray.
The Dermochelyidae family of turtles — more commonly known as "Leatherback sea turtle" — were first discovered in 1843 by Fitzinger.
The Emydidae family of turtles — more commonly known as "Pond or water turtles" — were first discovered in 1815 by Rafinesque.
The Geoemydidae family of turtles — more commonly known as "Asian river, leaf, roofed or Asian box turtles" — were first discovered in 1868 by Theobald.
The Kinosternidae family of turtles — more commonly known as "Mud or musk turtles" — were

we get the discoverer man

In [82]:
discoverer = []

for i in df['turtle text']:
    
    texto_sep = i.split('by')
    discoverer.append(texto_sep[-1].strip())

print(discoverer)
    


['Boulenger.', 'Oppel.', 'Gray.', 'Gray.', 'Fitzinger.', 'Rafinesque.', 'Theobald.', 'Agassiz.', 'Gray.', 'Batsch.', 'Fitzinger.', 'Gray.', 'Cope.', 'Gray.']


In [110]:
df['discoverer man'] = discoverer

In [112]:
df['discoverer man'] = df['discoverer man'].replace('\.', '',regex=True)

In [113]:
df

,cientific name,common name,turtle text,photo,discoverer man,discovered year
0,Carettochelyidae,Pig-nosed turtle,The Carettochelyidae family of turtles — more ...,https://upload.wikimedia.org/wikipedia/commons...,Boulenger,1887
1,Cheloniidae,Sea turtles,The Cheloniidae family of turtles — more commo...,https://upload.wikimedia.org/wikipedia/commons...,Oppel,1811
2,Chelydridae,Snapping turtles,The Chelydridae family of turtles — more commo...,https://upload.wikimedia.org/wikipedia/commons...,Gray,1831
3,Dermatemydidae,Central American river turtle,The Dermatemydidae family of turtles — more co...,https://upload.wikimedia.org/wikipedia/commons...,Gray,1870
4,Dermochelyidae,Leatherback sea turtle,The Dermochelyidae family of turtles — more co...,https://upload.wikimedia.org/wikipedia/commons...,Fitzinger,1843
5,Emydidae,Pond or water turtles,The Emydidae family of turtles — more commonly...,https://upload.wikimedia.org/wikipedia/commons...,Rafinesque,1815
6,Geoemydidae,"Asian river, leaf, roofed or Asian box turtles",The Geoemydidae family of turtles — more commo...,https://upload.wikimedia.org/wikipedia/commons...,Theobald,1868
7,Kinosternidae,Mud or musk turtles,The Kinosternidae family of turtles — more com...,https://upload.wikimedia.org/wikipedia/commons...,Agassiz,1857
8,Platysternidae,Big-headed turtle,The Platysternidae family of turtles — more co...,https://upload.wikimedia.org/wikipedia/commons...,Gray,1869
9,Testudinidae,Tortoises,The Testudinidae family of turtles — more comm...,https://upload.wikimedia.org/wikipedia/commons...,Batsch,1788


we get the discovered year

In [101]:
discovered_year = []

for i in df['turtle text']:
    
    texto_sep = i.split('by')
    first_phrase= texto_sep[0]

    texto_sep_spaces = first_phrase.split(' ')
    year = texto_sep_spaces[-2]

    discovered_year.append(year.strip())

print(discovered_year)
    


['1887', '1811', '1831', '1870', '1843', '1815', '1868', '1857', '1869', '1788', '1826', '1831', '1868', '1869']


In [102]:
df['discovered year'] = discovered_year

set the images in the dataframe

In [138]:
def display_image(url, width=200, height=200):
    return f'<img src="{url}" style="min-width:{width}px; min-height:{height}px; max-width:{width}px; max-height:{height}px;">'


df['image photo'] = df['photo'].apply(display_image)


In [139]:
df_photos = df.loc[:,['common name','cientific name','discoverer man','discovered year','image photo']]

### The dataset of turtles

In [140]:
display(HTML(df_photos.to_html(escape=False)))

,common name,cientific name,discoverer man,discovered year,image photo
0,Pig-nosed turtle,Carettochelyidae,Boulenger,1887,
1,Sea turtles,Cheloniidae,Oppel,1811,
2,Snapping turtles,Chelydridae,Gray,1831,
3,Central American river turtle,Dermatemydidae,Gray,1870,
4,Leatherback sea turtle,Dermochelyidae,Fitzinger,1843,
5,Pond or water turtles,Emydidae,Rafinesque,1815,
6,"Asian river, leaf, roofed or Asian box turtles",Geoemydidae,Theobald,1868,
7,Mud or musk turtles,Kinosternidae,Agassiz,1857,
8,Big-headed turtle,Platysternidae,Gray,1869,
9,Tortoises,Testudinidae,Batsch,1788,
